### 1- Apply Savings algorithm for the VRP (note that this algorithm should be rewritten for the VRP and not TSP and the data is asymmetric)

Importing necessary libraries

In [7]:
import pandas as pd
import numpy as np

Read the distance matrix & demand from the excel file Exam-Data.xls, I used the following link to learn how to do it: https://stackoverflow.com/questions/26521266/using-pandas-to-pd-read-excel-for-multiple-worksheets-of-the-same-workbook#:~:text=xls%20%3D%20pd.ExcelFile(%27path_to_file.xls%27)%0Adf1%20%3D%20pd.read_excel(xls%2C%20%27Sheet1%27)%0Adf2%20%3D%20pd.read_excel(xls%2C%20%27Sheet2%27)

In [8]:
#To read different sheets in an excel file we can you this block of code
xls = pd.ExcelFile('real_distances_30_customers.xlsx') 
df_distance_matrix = pd.read_excel(xls, 'distance', header=None) # header=None make first row not the header
df_demand = pd.read_excel(xls, 'demand', index_col=0) # col= 0 because in data we have the indexes

In [9]:
d = df_distance_matrix.values.tolist() # to make pandas df a python's built-in list

"demand = demand_df.values.tolist()" is gives the demand list in lists to make it not lists in a list I used following link, https://stackoverflow.com/questions/716477/join-list-of-lists-in-python#:~:text=x%20%3D%20%5B%5B%22a%22%2C%22b%22%5D%2C%20%5B%22c%22%5D%5D%0A%0Aresult%20%3D%20sum(x%2C%20%5B%5D)

In [405]:
demand_list = sum(df_demand.values.tolist(), [])  # explained above 

In [11]:
vehicle_capacity = 120 #given in the data 

In [12]:
origin = 0 # depot node

In [13]:
n_veh = int(np.ceil(sum(demand_list)/vehicle_capacity)) # min number of vehicle that we will use

In [14]:
n_veh

3

In [410]:
nodes = []
for i in range(31):
    nodes.append(i)
    
def VRP_savings(nodes, origin, d, n_veh, demand_list, vehicle_capacity):
    '''
    Constructs a VRP solution using the savings method for a given set/list of 
    nodes, their pairwise distances-d, the origin, number of vehicle, demand list and the vehicle capacity.
    '''
    ans = [] # Creating an empty list to store solution data
    for iteration in range(n_veh): # every iteration is for finding the vehicle's route and the tour length of the route
            # Set of customer nodes (i.e. nodes other than the origin)
            customers = {i for i in nodes if i != origin}

            # Initialize out-and-back tours from the origin to every other node
            tours = {(i,i): [origin, i, origin] for i in customers}

            # Compute savings
            savings = {(i, j): round(d[i][origin] + d[origin][j] - d[i][j], 2) 
                       for i in customers for j in customers if j != i}

            # Define a priority queue dictionary to get a pair of nodes (i,j) which yields
            # the maximum savings
            #pq = pqdict(savings, reverse = True)
            sorted_savings = sorted(savings.items(), key=lambda item: item[1]) # the savings in ASC order
            # Merge subtours until obtaining a TSP tour
            break_while = False # to control while loop
            while len(tours) > 1 and not break_while: #if you have only 1 tour left then its last vehicle's tour no need to check
                A = sorted_savings.pop() # pops the maximum savings(because sorted_savings in ASC order) and stores it as a variable 
                # A = (i, j) --> biggest saving
                i = A[0][0] # i
                j = A[0][1] # j
      
                break_outer = False  # Outer loop

                for t1 in tours: # iterate all tours 
                    for t2 in tours.keys()-{t1}: # iterate over all tours except t1
                        sum_= 0
                        for x in (tours[t1][:-1] + tours[t2][1:]): # iterates over the tour if merge happens 
                            sum_ += demand_list[x] # to find capacity will if merge happens
                        if sum_ > vehicle_capacity: # if used capacity > vehicle capacity
                            break_while = True  
                            break
                        if t1[1] == i and t2[0] == j:  # checks the constraint 𝑖 is the last node in its subtour and 𝑗 is the first node in its subtour (i.e., (𝑖,𝑜) and (𝑜,𝑗) are in the subtours containing 𝑖 and 𝑗, respectively)                     
                            tours[(t1[0], t2[1])] = tours[t1][:-1] + tours[t2][1:] # merging the tours
                            del tours[t1], tours[t2] # delete the tours (e.g. 0-1-0, 0-2-0 merged 0-1-2-0. now we have to delete 0-1-0 and 0-2-0)
                            break_outer = True 
                            break
                    if break_outer:
                        break 

            x = tours.popitem() # delete the tour that achieved its capacity or that will exceed its capacity if one more merge happens
            
            ans.append(x[1]) # append the value of that tour to ans list
            for i in ans[iteration][1:]: # delete the nodes that used 
                for j in nodes:
                    if i == j:
                        nodes.remove(i)
                        
    # compute the answer's length
    ans_len = 0 
    for r in ans:
        for i in range(len(r)-1):
            ans_len += d[r[i]][r[i+1]]
            
    return ans, round(ans_len, 2) # return the ans and ans_len

In [411]:
vrp_solution_saving, vrp_solution_saving_len = VRP_savings(nodes, origin, d, n_veh, demand_list, vehicle_capacity)
print(vrp_solution_saving, vrp_solution_saving_len)

[[0, 16, 13, 30, 6, 25, 7, 12, 21, 8, 18, 3, 0], [0, 22, 17, 5, 11, 10, 20, 15, 2, 29, 28, 26, 1, 0], [0, 27, 14, 4, 19, 24, 23, 9, 0]] 2542.2


### 2- Apply 2-exchange algorithms exhaustively (try all possible improvements) to improve the solution you obtained from 1.

I used list() otherwise in the swapList function it swaps all list that intended or unintended. I found the solution in that link:
https://stackoverflow.com/questions/29785084/changing-one-list-unexpectedly-changes-another-too#:~:text=different%20arrays%20use%3A-,vec%20%3D%20list(v),-Share

In [359]:
def two_exchange(vrp_solution, d):
    '''
    Improves a given VRP solution using the 2-exchange algorithm.
    It is a general function which can solve asymmetric and symmetric matrices. 
    '''
    
    def swapList(sl,pos1,pos2):
        '''
        Swaps 2 list element according to their index.
        The list, and indexes of 2 item that will be swapped must be given. 
        '''
        n = len(sl) # n --> length of the list
        
        # Swapping 
        temp = sl[pos1]
        sl[pos1] = sl[pos2]
        sl[pos2] = temp
        return sl  
    
    def sol_leng(list, d):
        '''
        gives the route length of a list.
        a list and a distance matrix must be given
        '''
        sol_len= 0
        for i in range(len(list)-1):
            sol_len += d[list[i]][list[i+1]]
        return sol_len
     
    improved_sol = [] # the list that we will be store data of the solution
    improved_sol_length = 0 # with that var we keep track of the length of the soloution
    for r in range(len(vrp_solution)): # iterates the indexes in vrp_solution that given
        sol = list(vrp_solution[r]) # creating the list to compare
        sol_len = sol_leng(sol, d) # giving the length value to compare

        for i in range(len(sol)): # iterating the indexes of the sol
            if sol[i] != 0: # if sol is not the origin node
                #print(sol)
                #print(sol_len)
                for j in range(len(sol)): # iterating the indexes of the sol
                    if sol[i] != sol[j] and sol[j] != 0: # if they are not the same or the origin node
                        # Making the exchanged list 
                        temp = list(sol) 
                        temp = list(swapList(temp, i, j))
                        temp_len = sol_leng(temp, d)
                        #print("swapping", i," and ", j)
                        #print("temp: ", temp, "temp len: ", temp_len)
                        # To control if we have a better solution after exchange
                        if temp_len < sol_len: # If we have a better solution via exchange we will make the exchange
                            #print("solution improved")
                            sol_len = temp_len
                            sol = list(temp)
        improved_sol.append(sol) # append the improved solution to improved_sol
        improved_sol_length += sol_len # add the length of the tour 
            
    return improved_sol, improved_sol_length

In [402]:
improved_sol_2ex, improved_sol_length_2ex = two_exchange(vrp_solution_saving, d)
print(improved_sol_2ex, round(improved_sol_length_2ex, 2))

[[0, 16, 13, 30, 6, 25, 7, 21, 8, 12, 18, 3, 0], [0, 22, 17, 5, 11, 10, 20, 2, 15, 29, 28, 26, 1, 0], [0, 27, 14, 4, 19, 24, 23, 9, 0]] 2522.9


### 3- Apply 2-opt algorithms greedily and stop when the first improvement is recognised to improve the solution you obtained from 2.

##### greedy_two_opt is the modificated two_opt TSPlib function. We showed the parts that we changed with #Greedy comment and explained
#### I added greedy_two_opt to TSPlib and I made a function called VRP_greedy_two_opt in VRPlib with using greedy_two_opt

In [390]:
def greedy_two_opt(tour, tour_length, d):
    '''
    Improves a given TSP solution using the Greedy 2-opt algorithm.
    It is a general function which can solve asymmetric and symmetric matrices. 
    '''
    current_tour, current_tour_length = tour, tour_length 
    best_tour, best_tour_length = current_tour, current_tour_length 
    solution_improved = True #Greedy (If improved stop)
    improved = False
    while solution_improved and not improved:
        print()
        print('Attempting to improve the tour', current_tour, 
              'with length', current_tour_length)
        solution_improved = False
        
        for i in range(1, len(current_tour)-2):
            for j in range(i+1, len(current_tour)-1):
                difference = round((d[current_tour[i-1]][current_tour[j]]
                                  + d[current_tour[i]][current_tour[j+1]]
                                  - d[current_tour[i-1]][current_tour[i]]
                                  - d[current_tour[j]][current_tour[j+1]])
                                  + d[current_tour[j]][current_tour[i]]     # with the added last two codes we can
                                  - d[current_tour[i]][current_tour[j]], 2) # use this in also asymmetric matrixs
                
                print('Cost difference due to swapping', current_tour[i], 'and',
                      current_tour[j], 'is:', difference)
                
                if current_tour_length + difference < best_tour_length:
                    print('Found an improving move! Updating the best tour...')
                    
                    best_tour = current_tour[:i] + list(reversed(current_tour[i:j+1])) + current_tour[j+1:]
                    best_tour_length = round(current_tour_length + difference, 2)
                    
                    print('Improved tour is:', best_tour, 'with length',
                          best_tour_length)
                    improved = True #Greedy (The tour is improved)
                    solution_improved = True
                if improved: #Greedy (breaks the inner for loop)
                    break
            if improved: #Greedy (breaks the outer for loop)
                break
        if improved: # Greedy (breaks the while loop)
            break
        current_tour, current_tour_length = best_tour, best_tour_length
    
    # Return the resulting tour and its length as a tuple
    return best_tour, best_tour_length

In [391]:
greedy_vrp_solution = [] # to store vrp route data
greedy_total_length = 0 # to count total length
for tour in improved_sol: # iterates over improved_sol that we found in 2-exchange algorithm 
    tour_length = sum(d[tour[i]][tour[i+1]] for i in range(len(tour)-1)) # computes total length of current tour
    new_tour, new_tour_length = greedy_two_opt(tour, tour_length, d) # using greedy_two_opt func that we add to TSPlib
    greedy_vrp_solution.append(new_tour) # adds data to greedy_vrp_solution
    greedy_total_length += new_tour_length # counts total length


Attempting to improve the tour [0, 16, 13, 30, 6, 25, 7, 21, 8, 12, 18, 3, 0] with length 1000.4000000000001
Cost difference due to swapping 16 and 13 is: 36.3
Cost difference due to swapping 16 and 30 is: 133.0
Cost difference due to swapping 16 and 6 is: 156.7
Cost difference due to swapping 16 and 25 is: 132.1
Cost difference due to swapping 16 and 7 is: 162.5
Cost difference due to swapping 16 and 21 is: 242.5
Cost difference due to swapping 16 and 8 is: 253.1
Cost difference due to swapping 16 and 12 is: 250.6
Cost difference due to swapping 16 and 18 is: 187.4
Cost difference due to swapping 16 and 3 is: 17.2
Cost difference due to swapping 13 and 30 is: 96.7
Cost difference due to swapping 13 and 6 is: 120.4
Cost difference due to swapping 13 and 25 is: 114.5
Cost difference due to swapping 13 and 7 is: 130.7
Cost difference due to swapping 13 and 21 is: 250.7
Cost difference due to swapping 13 and 8 is: 256.0
Cost difference due to swapping 13 and 12 is: 279.2
Cost difference 

In [401]:
print(greedy_vrp_solution, round(greedy_total_length, 2))

[[0, 16, 13, 30, 6, 25, 7, 21, 8, 12, 18, 3, 0], [0, 22, 17, 5, 11, 10, 20, 2, 15, 29, 28, 26, 1, 0], [0, 23, 24, 19, 4, 14, 27, 9, 0]] 2521.1
